In [1]:
import numpy as np
import xarray as xr
import pandas as pd
from metpy.units import units
import metpy.constants as mpconst
#import metpy.calc as mpcalc
from glob import glob
import gc
import holoviews as hv
import hvplot.xarray
import cartopy.crs as ccrs
import geoviews as gv
import matplotlib.ticker as tkr
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
hv.extension('bokeh')
#%matplotlib inline

### Column integrate a variable that has nan values

In [2]:
def mass_weighted_vertical_integral_w_nan(variable_to_integrate, pressure_model_level_midpoint_Pa, pressure_model_level_interface_Pa, max_pressure_integral_array_Pa, min_pressure_integral_array_Pa):
    
    # Accepts both integers and arrays as min and max pressure limits
    
    # Define constants
    
    g = 9.8 # [m s^-2]
    
    # Set all model interfaces less than minimum pressure equal to minimum pressure, and more than maximum pressure to maximum pressure. This way, when you calculate "dp", these layers will not have mass.
    
    pressure_model_level_interface_Pa = pressure_model_level_interface_Pa.where(pressure_model_level_interface_Pa < max_pressure_integral_array_Pa, other = max_pressure_integral_array_Pa)
    pressure_model_level_interface_Pa = pressure_model_level_interface_Pa.where(pressure_model_level_interface_Pa > min_pressure_integral_array_Pa, other = min_pressure_integral_array_Pa)

    # Calculate delta pressure for each model level
    
    dp = pressure_model_level_midpoint_Pa.copy()
    dp.values = xr.DataArray(pressure_model_level_interface_Pa.isel(ilev = slice(1, len(pressure_model_level_interface_Pa.ilev))).values - pressure_model_level_interface_Pa.isel(ilev = slice(0, -1)).values) # Slice indexing is (inclusive start, exclusive stop)
    
    # Set dp = nan at levels missing data so mass of those levels not included in calculation of dp_total
    
    dp = dp.where(~xr.ufuncs.isnan(variable_to_integrate), drop=False, other=np.nan)

    # Mass weight each layer
    
    ci_variable = variable_to_integrate * dp / g
    
    # Integrate over levels
    
    ci_variable = ci_variable.sum('lev', min_count=1)
    dp_total = dp.sum('lev', min_count=1)
    
    # Set ci_variable to nan wherever dp_total is zero or nan
    
    ci_variable = ci_variable.where(~(dp_total==0), drop = False, other=np.nan)
    ci_variable = ci_variable.where(~xr.ufuncs.isnan(dp_total), drop = False, other=np.nan)
    
    # Calculate mass weighted vertical average over layer integrated over
    
    mwa_variable = ci_variable * g / dp_total
    
    return ci_variable, dp_total, mwa_variable

In [6]:
def haversine(lat1, lon1, lat2, lon2, R=6371.0):
    lat1, lon1, lat2, lon2 = np.radians([lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c  # Earth radius in kilometers
    return distance * 1000  # Convert to meters


# Define input directories and file names
<br>
h0:  monthly mean  (1 time step per monthly file)
<br>
h1:  daily mean  (30 time steps per 30-day file)
<br>
h2:  6-hourly  (120 time steps per 30-day file)
<br>
h3:  3-hourly  (240 time steps per 30-day file)
<br>
h4:  hourly  (240 time steps per 10-day file)

In [3]:
# Years to analyze
start_year = (1980)
end_year = (1980)

################
###.  ERA5.  ###
################

# Atmosphere

input_file_string_specific_humidity = '/Projects/era5_regrid/2p5/shum.2p5.*.nc' # Specific humidity
input_file_string_temperature = '/Projects/era5_regrid/2p5/air.2p5.*.nc' # Temperature
input_file_string_surface_pressure = '/Projects/era5_regrid/2p5/pres.sfc.2p5.*.nc' # Surface Pressure
input_file_string_v_wnd = '/Projects/era5_regrid/2p5/vwnd.2p5.*.nc' # v wind
input_file_string_u_wnd = '/Projects/era5_regrid/2p5/uwnd.2p5.*.nc' # u wind
input_file_string_hgt = '/Projects/era5_regrid/2p5/hgt.2p5.*.nc' # geopotential height
input_file_string_omega = '/Projects/era5_regrid/2p5/omega.2p5.*.nc' #vertical velocity

# Land
input_file_string_land_frac = '/glade/work/bwolding/Datasets/Data_for_Glade/ERAi/land_sea_mask.erai.2p5.nc' # Land Fraction 

### Load in data and calculations

In [42]:
# Define constants

g = 9.8 #[m s^-2]
L = 2.26e6 #[J/kg]
cp = 1005 #[J/kg-K]

#########################################
# Define paths of files we wish to load #
#########################################
    
# glob expands paths with * to a list of files, like the unix shell #

paths_specific_humidity = glob(input_file_string_specific_humidity)
paths_temperature = glob(input_file_string_temperature)
paths_surface_pressure = glob(input_file_string_surface_pressure)
paths_v_wnd = glob(input_file_string_v_wnd)
paths_u_wnd = glob(input_file_string_u_wnd)
paths_hgt = glob(input_file_string_hgt)
paths_omega = glob(input_file_string_omega)


for year in range(start_year, end_year + 1):
        
    print(year)
    
    # Define year strings #
        
    current_year_string = str(year)
            
    # Limit paths #
        
    year_limited_paths_specific_humidity = []
    year_limited_paths_temperature = []
    year_limited_paths_u_wnd = []
    year_limited_paths_v_wnd = []
    year_limited_paths_hgt = []
    year_limited_paths_omega = []
        
    for string in paths_specific_humidity:
                        
        if (current_year_string in string):
                
            year_limited_paths_specific_humidity += [string]
            
    for string in paths_temperature:
                        
        if (current_year_string in string):
                
            year_limited_paths_temperature += [string]
            
    for string in paths_u_wnd:
                        
        if (current_year_string in string):
                
            year_limited_paths_u_wnd += [string]

    for string in paths_v_wnd:
                        
        if (current_year_string in string):
                
            year_limited_paths_v_wnd += [string]

    for string in paths_hgt:
                        
        if (current_year_string in string):
                
            year_limited_paths_hgt += [string]

    for string in paths_omega:
                        
        if (current_year_string in string):
                
            year_limited_paths_omega += [string]




#####################
####  Load Data  ####
#####################

# Data is "lazy loaded", nothing is actually loaded until we "look" at data in some way #

dataset_specific_humidity = xr.open_mfdataset(year_limited_paths_specific_humidity, combine="by_coords")
dataset_temperature = xr.open_mfdataset(year_limited_paths_temperature, combine="by_coords")
dataset_u_wnd = xr.open_mfdataset(year_limited_paths_u_wnd, combine="by_coords")
dataset_v_wnd = xr.open_mfdataset(year_limited_paths_v_wnd, combine="by_coords")
dataset_hgt = xr.open_mfdataset(year_limited_paths_hgt, combine="by_coords")
dataset_omega = xr.open_mfdataset(year_limited_paths_omega, combine="by_coords")

#####################
####  Load Data  ####
#####################
              
# Make data arrays, loading only the year of interest #
full_lat = dataset_temperature['lat']
full_lon = dataset_temperature['lon']


#PS = dataset_surface_pressure['pres'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'), lat = slice(15, -15)) # [Pa]
Q = dataset_specific_humidity['shum'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [Kg/Kg]
T = dataset_temperature['air'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]
U_wnd = dataset_u_wnd['uwnd'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]
V_wnd = dataset_v_wnd['vwnd'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]
HGT = dataset_hgt['hgt'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]
OMEGA = dataset_omega['omega'].sel(time = slice(str(year)+'-01-01', str(year)+'-12-31'),lat = slice(15, -15), level = slice(70, 1000)) # [K]

# Actually load data #

Q.load()
T.load()
U_wnd.load()
V_wnd.load()
HGT.load()
OMEGA.load()




1980


<xarray.DataArray 'omega' (time: 2928, level: 28, lat: 13, lon: 144)>
array([[[[-4.08852100e-03, -9.45961475e-03, -1.40982866e-02, ...,
           6.50107861e-03, -5.73647022e-03, -3.23402882e-03],
         [ 9.82749462e-03,  4.58598137e-04, -1.01004839e-02, ...,
           1.02852583e-02,  9.64438915e-03,  1.10481977e-02],
         [-6.77406788e-03, -5.76698780e-03, -2.37953663e-03, ...,
          -2.10487843e-03,  1.16890669e-02, -8.53657722e-04],
         ...,
         [ 2.76192427e-02, -1.19315386e-02,  1.00791454e-03, ...,
          -2.28873491e-02, -8.29994678e-03,  3.23569775e-03],
         [ 1.49619579e-03,  2.35068798e-03, -2.22769976e-02, ...,
          -1.47696733e-02, -1.38846636e-02,  8.48472118e-03],
         [-8.78822803e-03, -5.58388233e-03, -1.06728077e-03, ...,
          -2.64273882e-02,  1.78231001e-02,  1.55342817e-02]],

        [[-7.07817078e-03,  1.29108429e-02,  1.35517120e-02, ...,
          -2.19402313e-02,  3.98635864e-04, -4.11796570e-03],
         [-8.83102417e-04,  1.54495239e-04, -7.44438171e-03, ...,
           1.40571594e-03,  2.15530396e-04,  1.16596222e-02],
         [ 3.08246613e-02, -2.62260437e-03, -1.86748505e-02, ...,
          -9.82475281e-03, -1.40361786e-02, -3.20243835e-03],
...
           8.28685760e-02, -3.72486115e-02,  4.03881073e-02],
         [ 1.20677948e-02,  5.11302948e-02,  4.60033417e-02, ...,
           7.82299042e-02,  7.77416229e-02,  3.40404510e-02],
         [ 4.60033417e-02,  1.69506073e-02,  5.25951385e-02, ...,
           2.57396698e-02,  7.70092010e-02,  3.03459167e-03]],

        [[ 1.84507370e-02,  1.01499557e-02,  5.01890182e-02, ...,
           1.60503387e-03,  5.02300262e-03,  3.35874557e-02],
         [ 2.82573700e-03, -8.89301300e-03,  3.60288620e-02, ...,
          -7.42816925e-03, -3.08656693e-02,  1.79624557e-02],
         [-3.76605988e-03,  2.18687057e-02,  1.67417526e-02, ...,
          -7.42816925e-03, -1.79262161e-02,  1.35679245e-02],
         ...,
         [ 3.70054245e-02,  1.86948776e-02,  3.09019089e-02, ...,
           4.48179245e-02, -2.54535675e-03,  2.28452682e-02],
         [ 1.57651901e-02,  2.67515182e-02,  2.52866745e-02, ...,
           4.01792526e-02,  3.72495651e-02,  2.23569870e-02],
         [ 2.72397995e-02,  1.67417526e-02,  3.06577682e-02, ...,
           1.45444870e-02,  4.16440964e-02,  1.28355026e-02]]]],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 1980-12-31T21:00:00
  * level    (level) float64 70.0 100.0 125.0 150.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) float32 15.0 12.5 10.0 7.5 5.0 ... -7.5 -10.0 -12.5 -15.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5
Attributes: (12/15)
    GridType:                      Cylindrical Equidistant Projection Grid
    datum:                         wgs84
    long_name:                     3-hourly Omega on Pressure Levels
    units:                         Pa/s
    var_desc:                      Omega (dp/dt)
    standard_name:                 vertical_air_velocity_expressed_as_tendenc...
    ...                            ...
    parent_stat:                   Other
    number_of_significant_digits:  7
    ecmwf_local_table:             128
    ecmwf_parameter:               135
    actual_range:                  [-21.9917     8.492116]
    remap:                         remapped via ESMF_regrid_with_weights: Bil...

In [43]:
MSE = cp*T + L*Q + g*HGT

MSE

<xarray.DataArray (time: 2928, level: 28, lat: 13, lon: 144)>
array([[[[383534.28, 383939.56, 383074.88, ..., 384860.56, 384547.2 ,
          383905.9 ],
         [381595.5 , 381563.25, 380609.9 , ..., 381825.06, 382514.12,
          382179.06],
         [380909.38, 379728.94, 378964.6 , ..., 380379.06, 380913.2 ,
          381101.7 ],
         ...,
         [378512.5 , 378502.34, 379354.56, ..., 380906.3 , 381844.2 ,
          380876.97],
         [378686.88, 379591.34, 379105.53, ..., 383465.06, 383554.03,
          381530.75],
         [380693.75, 379771.62, 377657.97, ..., 386845.8 , 385789.25,
          381751.5 ]],

        [[357509.8 , 357800.88, 358485.  , ..., 358921.  , 357747.94,
          358108.94],
         [356326.4 , 357238.75, 356879.  , ..., 356921.84, 356170.3 ,
          354861.06],
         [356191.2 , 355702.25, 355147.56, ..., 354774.47, 354645.25,
          354879.47],
...
         [325155.2 , 326403.94, 326446.44, ..., 328854.84, 328869.3 ,
          327464.44],
         [326343.53, 325492.22, 325224.38, ..., 327935.47, 327261.28,
          326530.56],
         [324374.97, 323799.56, 323403.4 , ..., 325985.53, 325450.  ,
          325136.16]],

        [[307809.44, 308923.5 , 309054.53, ..., 305371.  , 305231.66,
          305492.6 ],
         [312016.88, 314622.22, 316469.34, ..., 310044.  , 311785.94,
          311701.4 ],
         [318891.12, 320587.22, 324478.84, ..., 316047.1 , 314020.47,
          314914.5 ],
         ...,
         [325287.3 , 326514.22, 326558.94, ..., 329279.28, 329098.03,
          327643.3 ],
         [326717.06, 325825.53, 325333.75, ..., 328355.44, 327370.66,
          326614.2 ],
         [324528.9 , 323898.72, 323487.28, ..., 326490.62, 326090.28,
          325560.12]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 1980-12-31T21:00:00
  * level    (level) float64 70.0 100.0 125.0 150.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) float32 15.0 12.5 10.0 7.5 5.0 ... -7.5 -10.0 -12.5 -15.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5

In [44]:
latitudes = MSE['lat'].values
longitudes = MSE['lon'].values

dMSE_dlat = MSE.differentiate('lat') / (5*100000)
dMSE_dlon = MSE.differentiate('lon') / (5*100000)

udMSE_dx = U_wnd*dMSE_dlon
vdMSE_dy = V_wnd*dMSE_dlat

HADV = udMSE_dx + vdMSE_dy

In [45]:
dMSE_dp = MSE.differentiate(coord= 'level')/100

VADV = OMEGA*dMSE_dp

VADV



<xarray.DataArray (time: 2928, level: 28, lat: 13, lon: 144)>
array([[[[ 3.54671963e-02,  8.24206397e-02,  1.15558363e-01, ...,
          -5.62117100e-02,  5.12443669e-02,  2.78093796e-02],
         [-8.27772915e-02, -3.71839013e-03,  7.98978806e-02, ...,
          -8.53786841e-02, -8.46899971e-02, -1.00604892e-01],
         [ 5.58142290e-02,  4.61872034e-02,  1.88911669e-02, ...,
           1.79648511e-02, -1.02349222e-01,  7.46160001e-03],
         ...,
         [-2.06177071e-01,  9.18483660e-02, -8.18291120e-03, ...,
           2.11012781e-01,  8.12528431e-02, -2.90377941e-02],
         [-1.20571256e-02, -1.88015848e-02,  1.71517804e-01, ...,
           1.51751623e-01,  1.39139503e-01, -7.61770606e-02],
         [ 7.23600760e-02,  4.49119806e-02,  7.46112643e-03, ...,
           2.98089385e-01, -1.80237204e-01, -1.27594218e-01]],

        [[ 4.34093066e-02, -8.03954080e-02, -8.25897679e-02, ...,
           1.30232036e-01, -2.44176574e-03,  2.56514624e-02],
         [ 5.01800748e-03, -8.98691942e-04,  4.21702266e-02, ...,
          -8.04812741e-03, -1.24002097e-03, -6.53063059e-02],
         [-1.77593157e-01,  1.40229529e-02,  9.40531716e-02, ...,
           5.28915524e-02,  7.61724189e-02,  1.75283067e-02],
...
           8.05389360e-02, -1.48987453e-02,  1.07212756e-02],
         [ 3.97716789e-03,  4.31463011e-02,  1.28150927e-02, ...,
           3.69230472e-02,  5.54006211e-02,  1.65419579e-02],
         [ 2.90189087e-02,  8.27846490e-03,  2.11402867e-02, ...,
           1.74625963e-02,  5.41182198e-02,  2.03461782e-03]],

        [[-5.99648920e-04,  6.55814027e-03, -6.12933422e-04, ...,
          -2.33131155e-04, -2.88697076e-04,  4.18583688e-04],
         [ 1.31054153e-03, -7.13553105e-04,  1.19052874e-02, ...,
          -1.18005753e-03, -1.88963488e-02,  8.94485414e-03],
         [-4.54280962e-04, -6.03986345e-03,  6.27187965e-03, ...,
           2.54916213e-03, -1.23023139e-02,  9.24569368e-03],
         ...,
         [ 1.95573666e-03,  8.24677758e-04,  1.39058591e-03, ...,
           7.60896364e-03, -2.32868333e-04,  1.63457904e-03],
         [ 2.35551642e-03,  3.56664648e-03,  1.10629201e-03, ...,
           6.74961181e-03,  1.62966840e-03,  7.47841201e-04],
         [ 1.67729065e-03,  6.64019724e-04,  1.02856813e-03, ...,
           2.93853204e-03,  1.06655732e-02,  2.17674067e-03]]]],
      dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 1980-12-31T21:00:00
  * level    (level) float64 70.0 100.0 125.0 150.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) float32 15.0 12.5 10.0 7.5 5.0 ... -7.5 -10.0 -12.5 -15.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5

In [46]:
# Specify the dimension representing pressure levels
pressure_dim = 'level'  # Replace with your dataset's dimension name

# Calculate the pressure spacing
pressure_levels = MSE[pressure_dim].values
pressure_spacing = np.diff(pressure_levels)*100/g  # Calculate the differences between adjacent pressure levels
pressure_spacing = np.insert(pressure_spacing, 0, pressure_spacing[0])  # Add the first spacing back for consistent dimensions

# Broadcast pressure_spacing to match the dimensions of data_variable
pressure_spacing_broadcasted = xr.DataArray(pressure_spacing, dims=pressure_dim)


# Compute the mass-weighted vertical integral
col_MSE = (MSE* pressure_spacing_broadcasted).sum(dim=pressure_dim)
col_HADV = (HADV* pressure_spacing_broadcasted).sum(dim=pressure_dim)
col_VADV = (VADV* pressure_spacing_broadcasted).sum(dim=pressure_dim)

pressure_spacing_broadcasted, col_HADV

#commenting to see what happens



(<xarray.DataArray (level: 28)>
 array([306.12244898, 306.12244898, 255.10204082, 255.10204082,
        255.10204082, 255.10204082, 255.10204082, 255.10204082,
        510.20408163, 510.20408163, 510.20408163, 510.20408163,
        510.20408163, 510.20408163, 510.20408163, 510.20408163,
        510.20408163, 510.20408163, 255.10204082, 255.10204082,
        255.10204082, 255.10204082, 255.10204082, 255.10204082,
        255.10204082, 255.10204082, 255.10204082, 255.10204082])
 Dimensions without coordinates: level,
 <xarray.DataArray (time: 2928, lat: 13, lon: 144)>
 array([[[ 1.92705103e+01,  1.85617379e+01,  1.76468774e+01, ...,
           2.27996492e+00,  1.50200624e+00,  2.42494688e+01],
         [ 7.12486895e+00,  1.07667083e+01,  3.50620849e+01, ...,
           2.85639535e+01,  6.37411305e-02,  1.17396188e+01],
         [ 8.90755057e+01,  4.96179581e+01,  2.41543765e+01, ...,
           2.62351826e+01,  5.10775991e+01,  6.11050492e+01],
         ...,
         [-8.85790585e+00, -1

In [28]:
filepath='/Projects/era5_regrid/2p5/omega.2p5.198001.nc'

test = xr.open_dataset(filepath)
test

MSE

<xarray.DataArray (time: 2928, level: 28, lat: 13, lon: 144)>
array([[[[202016.58, 202424.81, 201561.1 , ..., 203389.94, 203032.48,
          202383.72],
         [200093.9 , 200051.61, 199082.67, ..., 200330.52, 201013.56,
          200670.03],
         [199368.2 , 198216.7 , 197480.4 , ..., 198837.42, 199412.64,
          199569.53],
         ...,
         [196959.66, 196914.17, 197804.48, ..., 199152.67, 200118.67,
          199208.  ],
         [196987.4 , 197919.88, 197454.31, ..., 201595.16, 201738.55,
          199799.4 ],
         [198871.83, 197983.67, 195932.52, ..., 204886.88, 203973.6 ,
          199944.8 ]],

        [[196239.56, 196524.33, 197170.  , ..., 197693.55, 196522.19,
          196866.67],
         [194857.42, 195721.81, 195305.53, ..., 195499.34, 194706.31,
          193407.19],
         [194594.12, 194038.42, 193416.16, ..., 193157.05, 193007.78,
          193257.11],
...
         [321859.97, 323163.53, 323229.22, ..., 325504.4 , 325533.78,
          324134.2 ],
         [323015.38, 322213.12, 321987.2 , ..., 324537.03, 323860.06,
          323149.8 ],
         [320985.7 , 320458.3 , 320097.22, ..., 322504.4 , 321965.62,
          321695.53]],

        [[306416.56, 307582.6 , 307754.5 , ..., 303921.4 , 303765.88,
          304039.7 ],
         [310778.8 , 313427.62, 315306.16, ..., 308757.9 , 310465.25,
          310406.7 ],
         [317819.72, 319468.34, 323474.9 , ..., 314886.75, 312876.47,
          313765.8 ],
         ...,
         [324138.  , 325421.9 , 325493.84, ..., 328081.97, 327914.34,
          326462.62],
         [325530.8 , 324686.97, 324241.75, ..., 327104.16, 326115.34,
          325377.3 ],
         [323277.88, 322694.88, 322318.56, ..., 325152.4 , 324745.84,
          324258.72]]]], dtype=float32)
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 ... 1980-12-31T21:00:00
  * level    (level) float64 70.0 100.0 125.0 150.0 ... 925.0 950.0 975.0 1e+03
  * lat      (lat) float32 15.0 12.5 10.0 7.5 5.0 ... -7.5 -10.0 -12.5 -15.0
  * lon      (lon) float32 0.0 2.5 5.0 7.5 10.0 ... 350.0 352.5 355.0 357.5